**start**

In [511]:
import pandas as pd
import json
import numpy as np

with open('t20_wc_match_results.json') as f:
    data = json.load(f)

match_df = pd.DataFrame(data[0]['matchSummary'])
match_df.rename({'scorecard':'match_id'},axis=1,inplace=True)
best_11 = []

In [512]:
with open('t20_wc_batting_summary.json') as f:
    data = json.load(f)
    
    all_records_bat=[]
    for rec in data:
        all_records_bat.extend(rec['battingSummary'])

batting_df = pd.DataFrame(all_records_bat)

batting_df['out/not_out'] = batting_df.dismissal.apply(lambda x: "out" if len(x)>0 else "not_out" )
batting_df.drop(columns=['dismissal'],inplace=True)


In [513]:
batting_df['batsmanName'] = batting_df['batsmanName'].apply(lambda x: x.replace('â€', ''))
batting_df['batsmanName'] = batting_df['batsmanName'].apply(lambda x: x.replace('\xa0', ''))

In [514]:
match_ids_dict ={}

for index, row in match_df.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    match_ids_dict[key1] = row['match_id']
    match_ids_dict[key2] = row['match_id']

batting_df['match_id'] = batting_df.match.map(match_ids_dict)

In [515]:
batting_df['runs'] = pd.to_numeric(batting_df.runs, errors='coerce')
batting_df['balls'] = pd.to_numeric(batting_df.balls, errors='coerce')
batting_df['4s'] = pd.to_numeric(batting_df['4s'], errors='coerce')
batting_df['6s'] = pd.to_numeric(batting_df['6s'], errors='coerce')
batting_df['SR'] = pd.to_numeric(batting_df.SR, errors='coerce')

In [516]:
with open('t20_wc_bowling_summary.json') as f:
    data = json.load(f)
    all_records_bowl=[]
    for rec in data:
        all_records_bowl.extend(rec['bowlingSummary'])
bowling_df = pd.DataFrame(all_records_bowl)
bowling_df['match_id'] = bowling_df.match.map(match_ids_dict)


In [517]:
bowling_df['runs'] = pd.to_numeric(bowling_df.runs, errors='coerce')
bowling_df['overs'] = pd.to_numeric(bowling_df.overs, errors='coerce')
bowling_df['4s'] = pd.to_numeric(bowling_df['4s'], errors='coerce')
bowling_df['6s'] = pd.to_numeric(bowling_df['6s'], errors='coerce')
bowling_df['maiden'] = pd.to_numeric(bowling_df.maiden, errors='coerce')
bowling_df['wickets'] = pd.to_numeric(bowling_df['wickets'], errors='coerce')
bowling_df['economy'] = pd.to_numeric(bowling_df['economy'], errors='coerce')
bowling_df['0s'] = pd.to_numeric(bowling_df['0s'], errors='coerce')
bowling_df['wides'] = pd.to_numeric(bowling_df['wides'], errors='coerce')
bowling_df['noBalls'] = pd.to_numeric(bowling_df['noBalls'], errors='coerce')


In [518]:
bowling_df['bowlerName'] = bowling_df['bowlerName'].apply(lambda x: x.replace('â€', ''))
bowling_df['bowlerName'] = bowling_df['bowlerName'].apply(lambda x: x.replace('†', ''))
bowling_df['bowlerName'] = bowling_df['bowlerName'].apply(lambda x: x.replace('\xa0', ''))

In [519]:
with open('t20_wc_player_info.json') as f:
    data = json.load(f)

players_df = pd.DataFrame(data)

In [520]:
players_df['name'] = players_df['name'].apply(lambda x: x.replace('â€', ''))
players_df['name'] = players_df['name'].apply(lambda x: x.replace('†', ''))
players_df['name'] = players_df['name'].apply(lambda x: x.replace('\xa0', ''))


In [521]:
team_india = players_df[players_df.team == 'England']

**Batting : Openers**

In [522]:
# pos < 4 == t , innings > 3 == t , avg > 30 == t,  strikerate>140 , boundary percentage > 50%
import math
openers_df = batting_df[batting_df.battingPos <= 3]
openers_innings = batting_df['batsmanName'].value_counts()
openers_eligible = openers_innings[openers_innings > 3].index
openers_df = openers_df[openers_df['batsmanName'].isin(openers_eligible)]

helper_df = openers_df.groupby(['batsmanName','teamInnings']).agg({'runs':['sum','mean'],'balls': ['sum'],'4s':['sum'],'6s':['sum']}).reset_index()
final_list_dict = {
    'name' : [],
    'team' : [],
    'runs' :[],
    'avg' : [],
    'balls':[],
    'fours':[],
    'sixes':[],
    'strike_rate':[],
}
for index , row in helper_df.iterrows():
    runs_scored = row['runs']['sum']
    runs_avg = row['runs']['mean']
    ball_played = row['balls']['sum']
    fours = row['4s']['sum']*4
    sixes = row['6s']['sum']*6
    sr_avg = (runs_scored*100/ball_played)
    boundary_pc = ((fours+sixes)*100/runs_scored)
    decider = (sr_avg>140.00) & (runs_avg>30.00) & (boundary_pc>50.00)
    player_info = {
        'name':row.batsmanName[0],
         'team':row.teamInnings[0],
         'runs':runs_scored,
         'avg':math.ceil(runs_avg),
         'balls':ball_played,
         'fours':fours//4,
         'sixes':sixes//6,
         'strike_rate':sr_avg}
    
    if decider : 
        for i in final_list_dict:
            final_list_dict[i].append(player_info[i])
    
final_openers_df = pd.DataFrame(final_list_dict).sort_values(['avg','strike_rate'],ascending=False)
final_openers = final_openers_df.name.head(2).values
best_11.extend(final_openers)

**Batting : Middle-Order**

In [523]:
# pos > 2 == t , innings > 3 == t , avg > 40 == t,  strikerate>125 , avg_ball>20

anchors_df = batting_df[(batting_df.battingPos > 2) & (batting_df.battingPos < 6)]
anchors_innings = batting_df['batsmanName'].value_counts()
anchors_eligible = anchors_innings[anchors_innings > 3].index
anchors_df = anchors_df[anchors_df['batsmanName'].isin(anchors_eligible)]

helper_df = anchors_df.groupby(['batsmanName','teamInnings']).agg({'runs':['sum','mean'],'balls': ['sum','mean'],'4s':['sum'],'6s':['sum']}).reset_index()

final_list_dict = {
    'name' : [],
    'team' : [],
    'runs' :[],
    'avg_runs' : [],
    'balls':[],
    'avg_balls':[],
    'fours':[],
    'sixes':[],
    'strike_rate':[],
}
for index , row in helper_df.iterrows():
    runs_scored = row['runs']['sum']
    runs_avg = math.ceil(row['runs']['mean'])
    balls_played = row['balls']['sum']
    balls_avg = row['balls']['mean']
    fours = row['4s']['sum']*4
    sixes = row['6s']['sum']*6
    sr_avg = (runs_scored*100/balls_played)
    decider = (sr_avg>125) & (runs_avg >=40) & (balls_avg > 20)
    
    if decider : 
        player_info ={'name':row.batsmanName[0],
         'team':row.teamInnings[0],
         'runs':runs_scored,
         'avg_runs':(runs_avg),
         'balls':balls_played,
         'avg_balls':balls_avg,
         'fours': fours//4,
         'sixes': sixes//6,
         'strike_rate':sr_avg}
        for i in final_list_dict:
            final_list_dict[i].append(player_info[i])
    
final_anchors_df = pd.DataFrame(final_list_dict).sort_values(['avg_runs','strike_rate'],ascending=False)
final_anchors = final_anchors_df.name.head(3).values
best_11.extend(final_anchors)

**Lower Order**

In [524]:
# pos > 2 == t , innings > 3 == t , avg > 25 == t,  strikerate>130 , avg_ball>12, innings bowled>1

lower_df = batting_df[(batting_df.battingPos > 4)]
lower_bat_innings = batting_df['batsmanName'].value_counts()
lower_bowl_innings = bowling_df['bowlerName'].value_counts()

lower_batters = lower_bat_innings[lower_bat_innings > 3].index
lower_bowlers = lower_bowl_innings[lower_bowl_innings > 1].index
lower_player = [i for i in lower_batters] + [i for i in lower_bowlers]

lower_player_eligible = []
seen = set()

for item in lower_player:
    if item in seen:
        lower_player_eligible.append(item)
    else:
        seen.add(item)
        
# lower_df = lower_df[lower_df['batsmanName'].isin(lower_player_eligible)]
lower_bowling_df = bowling_df[bowling_df['bowlerName'].isin(lower_player_eligible)]
lower_batting_df = lower_df[lower_df['batsmanName'].isin(lower_player_eligible)]
lower_df = pd.merge(lower_batting_df,lower_bowling_df,left_on=['batsmanName','match_id','match','teamInnings'],right_on=['bowlerName','match_id','match','bowlingTeam'])
lower_df.drop(columns=['bowlerName','bowlingTeam'], inplace=True)
lower_df.rename(columns={'batsmanName':'name','teamInnings':'team'},inplace=True)

helper_df = lower_df.groupby(['name','team']).agg({'runs_x':['sum','mean'],'balls': ['sum','mean'],'4s_x':['sum'],'6s_x':['sum']}).reset_index()
helper_df.set_index('name',inplace=True)
final_list_dict = {
    'name' : [],
    'team' : [],
    'runs' :[],
    'avg_runs' : [],
    'balls':[],
    'avg_balls':[],
    'fours':[],
    'sixes':[],
    'strike_rate':[],
}
for index , row in helper_df.iterrows():
    runs_scored = row['runs_x']['sum']
    runs_avg = math.ceil(row['runs_x']['mean'])
    balls_played = row['balls']['sum']
    balls_avg = row['balls']['mean']
    fours = row['4s_x']['sum']*4
    sixes = row['6s_x']['sum']*6
    sr_avg = (runs_scored*100/balls_played)
    decider = (sr_avg>130) & (runs_avg >=25) & (balls_avg > 12)
    if decider : 
        player_info = {'name':index,
         'team':row.team[0],
         'runs':runs_scored,
         'avg_runs':(runs_avg),
         'balls':balls_played,
         'avg_balls':balls_avg,
         'fours': fours//4,
         'sixes': sixes//6,
         'strike_rate':sr_avg}
    
        for i in final_list_dict:
            final_list_dict[i].append(player_info[i])
    
final_lower_df = pd.DataFrame(final_list_dict).sort_values(['runs','strike_rate'],ascending=False)
final_lower = final_lower_df.name.head(2).values
best_11.extend(final_lower)

**All Rounders**

In [525]:
def cal_balls(overs):
    return int(int(overs)*6 + round(overs%1,1)*10)
def cal_overs(overs):
    final_overs = int(overs)
    balls = round(overs%1,1)*10
    while balls > 6:
        balls -= 6
        final_overs += 1
    if balls > 0:
        final_overs += balls/10;
    return final_overs

In [526]:
# pos > 4 == t , innings 2 , avg > 15,  strikerate>140 , innings bowled>2, economy < 7 , bowlingStrikeRate < 20
ar_df = batting_df[(batting_df.battingPos > 4)]
batsmans = ar_df.batsmanName.value_counts()
batters = batsmans[batsmans > 2].index

bowlers = bowling_df.bowlerName.value_counts()
bowlers = bowlers[bowlers > 2].index

all_rounders = [i for i in batters] + [i for i in bowlers]
ar_eligible = []
seen = set()

for item in all_rounders:
    if item in seen:
        ar_eligible.append(item)
    else:
        seen.add(item)

ar_batting_df = ar_df[ar_df.batsmanName.isin(ar_eligible)]
ar_bowling_df = bowling_df[bowling_df.bowlerName.isin(ar_eligible)]

ar_df = pd.merge(ar_batting_df,ar_bowling_df,left_on=['batsmanName','match','match_id','teamInnings'],right_on=['bowlerName','match','match_id','bowlingTeam'])
ar_df.drop(columns=['bowlerName','bowlingTeam'], inplace=True)
ar_df.rename(columns={'batsmanName':'name','teamInnings':'team'},inplace=True)


helper_df = ar_df.groupby(['name','team']).agg({'runs_x':['sum','mean'], 'balls':['sum'],'runs_y':['sum'],'overs':['sum'],'wickets':['sum']}).reset_index()
final_list_dict = {
    'name' : [],
    'team' : [],
    'runs_scored' :[],
    'avg_runs_scored' : [],
    'balls_faced':[],
    'strike_rate':[],
    'overs':[],
    'runs_got':[],
    'wickets':[],
    'economy':[],
}

for index,row in helper_df.iterrows():
    runs_scored = row['runs_x']['sum']
    runs_avg = math.ceil(row['runs_x']['mean'])
    balls_played = row['balls']['sum']
    bat_sr = (runs_scored*100/balls_played)
    runs_got = row['runs_y']['sum']
    overs = cal_overs(row['overs']['sum'])
    balls_bowled = cal_balls(overs)
    economy = runs_got/overs
    wickets=row['wickets']['sum']
    
    if wickets > 0:
        bowl_sr = balls_bowled/wickets
        decider = (bat_sr>140) & (runs_avg >= 15)&(bowl_sr <20)&(economy < 7)
        if decider:
            player_info = [
            {'name':row['name'][0],
             'team':row.team[0],
             'runs_scored':runs_scored,
             'avg_runs_scored':(runs_avg),
             'balls_faced':balls_played,
             'strike_rate':bat_sr,
             'overs':overs,
             'runs_got':runs_got,
             'wickets':wickets,
             'economy':economy,
            }
        ]
            for i in final_list_dict:
                final_list_dict[i].append(player_info[0][i])
    
final_ar_df = pd.DataFrame(final_list_dict).sort_values(['runs_scored','strike_rate','economy','wickets'],ascending=False)
final_players = list(final_ar_df.name.head(2))
for i in (final_players):
    if i in best_11 :
        final_players.remove(i)
best_11.extend(final_players)

**Fast Bowlers**

In [548]:
# innnigs bowled>4 T, style=%Fast% T,  economy<7,  bowl-sr<16,   bowl_avg < 20, 0s >40
bowlers = bowling_df.bowlerName.value_counts()
bowlers = bowlers[bowlers > 4].index

bowlers_info = players_df[players_df.name.isin(bowlers)& players_df.bowlingStyle.str.contains('Fast|fast')]
fast_bowlers = list(bowlers_info.name)
bowlers_df = bowling_df[bowling_df.bowlerName.isin(fast_bowlers)]
bowlers_df = bowlers_df.copy()
bowlers_df.rename(columns={'bowlerName':'name','bowlingTeam':'team'},inplace=True)

helper_df = bowlers_df.groupby(['name','team']).agg({'runs':'sum','overs':'sum','wickets':'sum','0s':'sum'}).reset_index()

final_list_dict = {
    'name':[],
    'team':[],
    'economy':[],
    'wickets':[],
    'dot_pc':[]
}

for index, row in helper_df.iterrows():
    runs = row['runs']
    wickets=row['wickets'],
    overs=cal_overs(row['overs'])
    balls=cal_balls(overs)
    dots_pc = round(row['0s']*100/balls,2)
    economy = round(runs/overs,2)
    
    if wickets[0] > 0:
        sr = balls/wickets[0]
        avg = runs/wickets[0]
        decider = (economy<7) & (sr < 16) & (avg < 20) & (dots_pc > 40)
        if decider:
            player_info = {
                'name' : row['name'],
                'team': row['team'],
                'economy':economy,
                'wickets':wickets[0],
                'dot_pc':dots_pc
            }
            for i in final_list_dict:
                final_list_dict[i].append(player_info[i])
                
final_ar = pd.DataFrame(final_list_dict).sort_values(['wickets','economy','dot_pc'],ascending=False)
final_players = final_ar.name.head(3).values
best_11.extend(final_players)
final_players

array(['Sam Curran', 'Shaheen Shah Afridi', 'Anrich Nortje'], dtype=object)

In [547]:
best_11_df = players_df[players_df.name.isin(best_11)]
best_11_df = best_11_df.copy()
best_11_df.drop(columns='description',inplace=True)
best_11_df

,name,team,battingStyle,bowlingStyle,playingRole
39,Sikandar Raza,Zimbabwe,Right hand Bat,Right arm Offbreak,Batting Allrounder
68,Jos Buttler(c),England,Right hand Bat,,Wicketkeeper Batter
77,Sam Curran,England,Left hand Bat,Left arm Medium fast,Allrounder
121,Shadab Khan,Pakistan,Right hand Bat,Legbreak,Allrounder
125,Shaheen Shah Afridi,Pakistan,Left hand Bat,Left arm Fast,Bowler
129,Virat Kohli,India,Right hand Bat,Right arm Medium,Top order Batter
130,Suryakumar Yadav,India,Right hand Bat,"Right arm Medium, Right arm Offbreak",Batter
132,Hardik Pandya,India,Right hand Bat,Right arm Medium fast,Allrounder
171,Anrich Nortje,South Africa,Right hand Bat,Right arm Fast,Bowler
176,Glenn Phillips,New Zealand,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter
